In [27]:
import pandas as pd
import os
import requests
from dotenv import load_dotenv
import time

In [26]:
# %pip install python-dotenv

In [7]:
load_dotenv()
RIOT_API_KEY = os.getenv("RIOT_API_KEY")

In [33]:
apex_tiers = ['challengerleagues', 'grandmasterleagues', 'masterleagues']
#divisions = ['DIAMOND', "EMERALD"]
divisions = ['DIAMOND']
tiers = ["I", "II", "III", "IV"]
#tiers = ["I"]

In [ ]:
#im just looking at the docs

In [9]:
#variable to input API key in the header of request instead of query
headers = {
    "X-Riot-Token" : RIOT_API_KEY
}

In [34]:
def get_summoner_ids_from_api(api_url, req_type):
    summoner_ids = []
    api_url = api_url
    try:
        resp = requests.get(api_url, headers=headers )  #Insert API link here
        resp.raise_for_status()
    except requests.HTTPError:
        print("Couldn't complete request")   
    response = resp.json()
    if req_type == "apex":
        entries = response['entries']
        for entry in entries:
            summoner_ids.append(entry['summonerId'])
    if req_type == "regular":
        for entry in response:
            summoner_ids.append(entry['summonerId'])
    return summoner_ids


def get_sums_for_apex_leagues():
    apex_summoner_ids = []
    for apex_league in apex_tiers:
        print(apex_league)
        api_url = f"https://na1.api.riotgames.com/lol/league/v4/{apex_league}/by-queue/RANKED_SOLO_5x5"
        summoner_ids = get_summoner_ids_from_api(api_url, "apex")
        apex_summoner_ids += summoner_ids
    return apex_summoner_ids

def get_sums_for_reg_divisions(start_time):
    inner_start_time = start_time
    total_loops = 3 #starts at 3 since there should be 3 apex tier calls before this
    reg_division_ids = []
    for division in divisions:
        print(division)
        for tier in tiers:
            print(tier)
            page = 1
            more_pages=True
            while more_pages:
                if total_loops == 100: #if we have made 100 requests
                    dif = time.time()-inner_start_time
                    if dif< 120: #if it has not been 120 seconds
                        print(f'sleeping for {dif}')
                        time.sleep(dif)
                        inner_start_time = time.time() #reset start time
                        total_loops = 0 #reset number of loops as well
                api_url = f'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{division}/{tier}?page={page}'
                output_list = get_summoner_ids_from_api(api_url, "regular")
                if output_list:
                    print(page)
                    page += 1
                    reg_division_ids += output_list
                else:
                    more_pages = False
                total_loops += 1 #keeping track of number of api calls for rate limiting purposes
    return reg_division_ids

def get_all_wanted_sums():
    all_sum_ids = []
    start = time.time()
    all_sum_ids += get_sums_for_apex_leagues()
    all_sum_ids += get_sums_for_reg_divisions(start)
    return all_sum_ids


#could later make this a dataframe the whole time, and add the rank information along with the summoner ID
# schema = {'summonerId': 'str', 'Division': 'str', 'Tier': 'str'}
# df = pd.DataFrame(columns=schema.keys()).astype(schema)

all_wanted_sum_ids = get_all_wanted_sums()
print(all_wanted_sum_ids)
print(len(all_wanted_sum_ids))
all_wanted_sum_ids = list(set(all_wanted_sum_ids))
print(len(all_wanted_sum_ids)) #see if it dropped any duplicates


challengerleagues
grandmasterleagues
masterleagues
DIAMOND
I
1
2
3
4
5
6
7
8
9
10
II
1
2
3
4
5
6
7
8
9
10
11
12
13
14
III
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
IV
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
['QkSyD-ZIMQz1G7X7lLe5547whcdvwA0LEQZwId022S0_eqUEbfbGy_5k5Q', '4e2zrrf9iztaVpsWnUF9HMuKdVcYQju-ihzU2cmSTAodTS8', 'rfxZxJfUv2AGZq5Gkm9i7IxBHgiu6f_2jW9p_R18mBcptPg', 'V9_TFwcm0O92518NTK9-CkBAgESP7uChQfZCn9LSOUCkZIE', 'O-47v7didTEApweyYFVcA8G5bObwSVeTKNcpoUk6yQjZ9FDza5aKkdvoSw', 'PchKnRxFWUOUZ-0xCghC7wsEX9SPnOHX1NfFzHdjB_dvonE', 'y5MAvhEkLtBuTbaze6sWyemkJSuSDIOURZBGOx-OuhIcRwWj', '5RhkzMBHpysywApF6tuWUeKYf_VoRMudhutumT_2N-NmOAc', 'CJgnWU0gLYP37D8arhHKRhHABzS4O9sVJxMeNhGCFPGI8dw', 'WgoiKnVfmynLxuPExGuOteMrOrGcMabevPFfT_9bYySQr4EmE7OQ64YFgQ', 'I5BMh_DpjxCdJIqyGiD1nXBbAU3aYr4K78vGysa8k5wweuXO7-xBHjhFxA', 'bEuM_rI_zvHk_MNxIOgJnl0Sjpjnk10j0hHNSo7nN0JiqTQ', '6CPNVj4ZHs0XCtEgKMVsfKc04Y_VNWAuz9IKMtt

,0
0,rlesU8V-exQlYK_vWUX1KMYJBlSYRMS2FjmFxcQtsOcTts...
1,N18WWKMoh2OqecKRydwlCFChsIOiOYUCg8VMdaNqIy1DPX...
2,Bjzd8iZc1Y2XFVYKEBIsreeVfcRlKPA_uiXwSG1Kb-LMLoXt
3,7kKrXfBU6qNqc61hfmxm8qKtnHdnE5Ja_9sVTGVQmKdKh8...
4,ghql6tGT5mflZdtwcs0M6bX_kA1pqDmNTzgFv47uZBCzLe...
...,...
20762,-PeZ-PVcrJY5WCHZ-Myid4hiXb1ZJcSAf1iEACa8Cuin-rw
20763,_aIvolE_GJAeXcsBjr6lBd7cOIL1Krqp9IQmCtgWYzBEHF...
20764,OBK1ECFg09uhzACsIWNzBsrmDbBVAzNrltkaMZePUTHpfd...
20765,JhQvKwjcFEyRBFjeRjPmkrx5-9LjeNzo3aSshbGot8e7FbM


In [40]:
sums_df = pd.DataFrame({"summonerId": all_wanted_sum_ids})
sums_df

,summonerId
0,rlesU8V-exQlYK_vWUX1KMYJBlSYRMS2FjmFxcQtsOcTts...
1,N18WWKMoh2OqecKRydwlCFChsIOiOYUCg8VMdaNqIy1DPX...
2,Bjzd8iZc1Y2XFVYKEBIsreeVfcRlKPA_uiXwSG1Kb-LMLoXt
3,7kKrXfBU6qNqc61hfmxm8qKtnHdnE5Ja_9sVTGVQmKdKh8...
4,ghql6tGT5mflZdtwcs0M6bX_kA1pqDmNTzgFv47uZBCzLe...
...,...
20762,-PeZ-PVcrJY5WCHZ-Myid4hiXb1ZJcSAf1iEACa8Cuin-rw
20763,_aIvolE_GJAeXcsBjr6lBd7cOIL1Krqp9IQmCtgWYzBEHF...
20764,OBK1ECFg09uhzACsIWNzBsrmDbBVAzNrltkaMZePUTHpfd...
20765,JhQvKwjcFEyRBFjeRjPmkrx5-9LjeNzo3aSshbGot8e7FbM


In [41]:
sums_df.to_csv("all_wanted_summoner_ids.csv")